<img src='https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcQ-VfNtOyJbsaxu43Kztf_cv1mgBG6ZIQZEVw&usqp=CAU'>

# Procesamiento de Lenguaje Natural

## Taller #9: Modelado de temas
`Fecha de entrega: Mayo 6, 2021. (Antes del inicio de la próxima clase).`

`Modo de entrega: Subir link de GitHub al aula virtual.`

In [1]:
import re
import pandas as pd 
from pprint import pprint

from nltk.corpus import stopwords
stopwords = stopwords.words('spanish')

import pyLDAvis.gensim_models
from gensim.models import LdaModel
from gensim.corpora import Dictionary

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

C:\Users\CHOMELY\anaconda3\lib\site-packages\scipy\sparse\sparsetools.py:21: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()


In [2]:
# Cargar datos
path = 'C:/Users/CHOMELY/Documents/1. Especialización Konrad Lorenz/1. Procesamiento de Lenguaje Natural (NLP)/Procesamiento-de-Lenguaje-Natural-NLP-/reviews_vidjew_es.csv'
data = pd.read_csv(path)
data.head()

,review_id,product_id,reviewer_id,stars,review_body,review_title,language,product_category
0,es_0825565,product_es_0370490,reviewer_es_0174781,3,"Buen. Buena calidad, y buena presentación.",Contenta,es,jewelry
1,es_0227934,product_es_0354224,reviewer_es_0411613,3,"Un producto a perfecto, para salir de casa con...",Versatilidad,es,video_games
2,es_0468601,product_es_0665460,reviewer_es_0348315,1,No funciona con Nintendo Switch. No hay forma ...,Decepción absoluta,es,video_games
3,es_0814494,product_es_0692692,reviewer_es_0951508,5,"Recomendado, los utilizo para pc y no me dan n...",Auriculares Pecham ps4,es,video_games
4,es_0206329,product_es_0728826,reviewer_es_0493255,4,El cable funciona bien podria ser un poco mas ...,Perfecto,es,video_games


###  `[12 pts]` Punto 1: Hacer pre-procesamiento del texto

In [3]:
print(f"Tenemos: {data.shape[0]} Registros.")

data.sample(5)

Tenemos: 1000 Registros.


,review_id,product_id,reviewer_id,stars,review_body,review_title,language,product_category
359,es_0135558,product_es_0731242,reviewer_es_0971288,5,buen regalo a precio bueno y envio muy rapido,precio ok,es,video_games
433,es_0804404,product_es_0714894,reviewer_es_0693912,5,Lo esperado. . Muy cómodo para poder jugar con...,Ideal,es,video_games
464,es_0774971,product_es_0299301,reviewer_es_0569765,2,El cierre no es muy seguro. Se me perdió. Por ...,Cierre no seguro,es,jewelry
146,es_0139546,product_es_0561652,reviewer_es_0921443,4,"A pesar de ser un juego con unos cuantos años,...",Un imprescindible en la ludoteca,es,video_games
720,es_0857929,product_es_0421209,reviewer_es_0732807,4,"El producto está muy bien, el problema es que ...",Bien,es,video_games


In [4]:
def pre_procesado(texto):
    texto = texto.lower()
    texto = re.sub(r"[\W\d]+", " ", texto)
    texto = [palabra for palabra in texto.split() if palabra not in stopwords]
    return texto 

data['pp'] = data['review_body'].apply(lambda texto: pre_procesado(texto)) #el modelo lda no viene incluido en Sklearn
#no recibe bien el TF IDF, se debe pasar una lista de palabras. El mismo autoprocesa los datos,

data.head()

,review_id,product_id,reviewer_id,stars,review_body,review_title,language,product_category,pp
0,es_0825565,product_es_0370490,reviewer_es_0174781,3,"Buen. Buena calidad, y buena presentación.",Contenta,es,jewelry,"[buen, buena, calidad, buena, presentación]"
1,es_0227934,product_es_0354224,reviewer_es_0411613,3,"Un producto a perfecto, para salir de casa con...",Versatilidad,es,video_games,"[producto, perfecto, salir, casa, nintendo, sw..."
2,es_0468601,product_es_0665460,reviewer_es_0348315,1,No funciona con Nintendo Switch. No hay forma ...,Decepción absoluta,es,video_games,"[funciona, nintendo, switch, forma, emparejarl..."
3,es_0814494,product_es_0692692,reviewer_es_0951508,5,"Recomendado, los utilizo para pc y no me dan n...",Auriculares Pecham ps4,es,video_games,"[recomendado, utilizo, pc, dan, ningún, proble..."
4,es_0206329,product_es_0728826,reviewer_es_0493255,4,El cable funciona bien podria ser un poco mas ...,Perfecto,es,video_games,"[cable, funciona, bien, podria, ser, mas, larg..."


,review_id,product_id,reviewer_id,stars,review_body,review_title,language,product_category,pp
0,es_0825565,product_es_0370490,reviewer_es_0174781,3,"Buen. Buena calidad, y buena presentación.",Contenta,es,jewelry,"[buen, buena, calidad, buena, presentación]"
1,es_0227934,product_es_0354224,reviewer_es_0411613,3,"Un producto a perfecto, para salir de casa con...",Versatilidad,es,video_games,"[producto, perfecto, salir, casa, nintendo, sw..."
2,es_0468601,product_es_0665460,reviewer_es_0348315,1,No funciona con Nintendo Switch. No hay forma ...,Decepción absoluta,es,video_games,"[funciona, nintendo, switch, forma, emparejarl..."
3,es_0814494,product_es_0692692,reviewer_es_0951508,5,"Recomendado, los utilizo para pc y no me dan n...",Auriculares Pecham ps4,es,video_games,"[recomendado, utilizo, pc, dan, ningún, proble..."
4,es_0206329,product_es_0728826,reviewer_es_0493255,4,El cable funciona bien podria ser un poco mas ...,Perfecto,es,video_games,"[cable, funciona, bien, podria, ser, mas, larg..."


###  `[10 pts]` Punto 2: Modelo de LDA

In [5]:
dictionary = Dictionary(data['pp'].values) # Crear una represantación de los documentos en forma de diccionario
dictionary #Diccionario de tipo gensim que el modelo va a utilizar


In [6]:
dictionary.filter_extremes(no_below=5, no_above=0.5) # Filtrar palabras muy frecuentes o infrecuentes filtrar extremos, número po debajo 
#igual a 5 y por enciam las palabras frecuente no agregan informacion valiosa para la clasificacion quite las palabras que 
#están en más del 50% de los documentso 0.5 y si no aparece si una palabra no aparece en más ede 5 documentos quitar 

In [7]:
# Corpus
corpus = [dictionary.doc2bow(text) for text in data['pp'].values] #Obtener el corpus bag o wors, de documento a bolsa de palabras
corpus #El corpus por cada documento entrega las palabras y su ubicacion la cantidad de veces que currieron en el documentso
#Una representacio vectorial

[[(0, 1), (1, 2), (2, 1), (3, 1)],
 [(4, 1), (5, 1), (6, 1), (7, 1), (8, 1)],
 [(5, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1)],
 [(13, 2), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1)],
 [(10, 1),
  (13, 1),
  (18, 1),
  (19, 1),
  (20, 1),
  (21, 1),
  (22, 1),
  (23, 1),
  (24, 1),
  (25, 1),
  (26, 1)],
 [(27, 1), (28, 1)],
 [(2, 1), (3, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1)],
 [(7, 3),
  (21, 1),
  (29, 1),
  (35, 1),
  (36, 1),
  (37, 1),
  (38, 2),
  (39, 1),
  (40, 1),
  (41, 1),
  (42, 1),
  (43, 1),
  (44, 1),
  (45, 1),
  (46, 1),
  (47, 1),
  (48, 1),
  (49, 1),
  (50, 1),
  (51, 1),
  (52, 2),
  (53, 1),
  (54, 1)],
 [(2, 1),
  (13, 1),
  (24, 1),
  (31, 1),
  (55, 1),
  (56, 1),
  (57, 1),
  (58, 1),
  (59, 1),
  (60, 1),
  (61, 1)],
 [(6, 1),
  (24, 1),
  (26, 1),
  (51, 1),
  (62, 1),
  (63, 1),
  (64, 1),
  (65, 1),
  (66, 1)],
 [(67, 1), (68, 1)],
 [(31, 1), (47, 1), (52, 1), (69, 1), (70, 1), (71, 1), (72, 1)],
 [(73, 1), (74, 1), (75, 1)],
 [(2,

In [30]:
# Entrenar el modelo
model = LdaModel(corpus = corpus, id2word = dictionary, num_topics = 5, passes=50) #id2word de indice a palabras, 7 número de temas
#Pasadas para que el modelo mejore

In [32]:
model.print_topics(num_words=10) #imprima los tema del modelo y muestre las 10 palabras más importante

[(0,
  '0.031*"bien" + 0.026*"si" + 0.020*"mas" + 0.020*"precio" + 0.018*"plata" + 0.016*"ps" + 0.015*"mejor" + 0.015*"mando" + 0.014*"mal" + 0.012*"original"'),
 (1,
  '0.040*"producto" + 0.032*"día" + 0.027*"llegado" + 0.020*"llego" + 0.019*"bien" + 0.018*"tiempo" + 0.018*"llegó" + 0.017*"amazon" + 0.016*"perfecto" + 0.015*"si"'),
 (2,
  '0.040*"bien" + 0.020*"bastante" + 0.018*"mando" + 0.016*"si" + 0.015*"así" + 0.014*"dos" + 0.014*"foto" + 0.014*"juegos" + 0.014*"cadena" + 0.013*"demasiado"'),
 (3,
  '0.086*"juego" + 0.019*"caja" + 0.019*"jugar" + 0.018*"calidad" + 0.017*"regalo" + 0.017*"buen" + 0.016*"producto" + 0.016*"bien" + 0.016*"buena" + 0.015*"gran"'),
 (4,
  '0.057*"calidad" + 0.040*"precio" + 0.027*"bonito" + 0.023*"buena" + 0.019*"pulsera" + 0.016*"bien" + 0.015*"regalo" + 0.013*"puede" + 0.013*"buen" + 0.013*"bonita"')]

###  `[25 pts]` Punto 3: Visualización de LDA

In [33]:
lda_display = pyLDAvis.gensim_models.prepare(model, corpus, dictionary, sort_topics=True) #sort topics ordenar
pyLDAvis.display(lda_display)

In [31]:
d = dictionary.doc2bow(["juego", "aretes", "pulsera","calidad"])
topics = [(cluster[0]+1, cluster[1]) for cluster in model.get_document_topics(d)]
topics

[(1, 0.050154116),
 (2, 0.050062824),
 (3, 0.05008083),
 (4, 0.33621055),
 (5, 0.5134917)]

In [34]:
def get_doc_top_n(text_processed, n):
    d = dictionary.doc2bow(text_processed)
    topics = dict(model.get_document_topics(d))
    try:
        return topics[n]
    except:
        return None

In [36]:
for t in range(0,7):
    top_name = f"topic_{t}"
    data[top_name] = data['pp'].apply(lambda doc: get_doc_top_n(doc, t))

In [37]:
data

,review_id,product_id,reviewer_id,stars,review_body,review_title,language,product_category,pp,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6
0,es_0825565,product_es_0370490,reviewer_es_0174781,3,"Buen. Buena calidad, y buena presentación.",Contenta,es,jewelry,"[buen, buena, calidad, buena, presentación]",0.033364,0.033437,0.033440,0.563852,0.336036,None,None
1,es_0227934,product_es_0354224,reviewer_es_0411613,3,"Un producto a perfecto, para salir de casa con...",Versatilidad,es,video_games,"[producto, perfecto, salir, casa, nintendo, sw...",0.429967,0.282718,0.219715,0.034027,0.033620,None,None
2,es_0468601,product_es_0665460,reviewer_es_0348315,1,No funciona con Nintendo Switch. No hay forma ...,Decepción absoluta,es,video_games,"[funciona, nintendo, switch, forma, emparejarl...",0.376907,0.028669,0.028911,0.537010,0.028573,None,None
3,es_0814494,product_es_0692692,reviewer_es_0951508,5,"Recomendado, los utilizo para pc y no me dan n...",Auriculares Pecham ps4,es,video_games,"[recomendado, utilizo, pc, dan, ningún, proble...",0.898768,0.025165,0.025770,0.025142,0.025149,None,None
4,es_0206329,product_es_0728826,reviewer_es_0493255,4,El cable funciona bien podria ser un poco mas ...,Perfecto,es,video_games,"[cable, funciona, bien, podria, ser, mas, larg...",0.327713,0.016911,0.349622,0.288595,0.016860,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,es_0427672,product_es_0899883,reviewer_es_0474735,2,En la foto parece que la cadena sea más gruesa...,Cadena muy fina,es,jewelry,"[foto, parece, cadena, gruesa, fina, precio, p...",0.022682,0.022432,0.909194,0.022370,0.023319,None,None
996,es_0059540,product_es_0702564,reviewer_es_0754753,3,"Bien por su precio, la cadena demasiado fina.",E,es,jewelry,"[bien, precio, cadena, demasiado, fina]",0.034447,0.033434,0.862661,0.033563,0.035990,None,None
997,es_0303349,product_es_0678671,reviewer_es_0172313,5,Muy bien!! Ahora tengo organizados los pendien...,Bonito,es,jewelry,"[bien, ahora, organizados, pendientes]",0.051018,0.050463,0.796546,0.050755,0.051177,None,None
998,es_0659961,product_es_0294943,reviewer_es_0554554,4,"Las tapas para los joystics no están nada mal,...",No está mal pero sólo he probado las fundas.,es,video_games,"[tapas, joystics, mal, ayudan, montón, joystic...",0.937790,0.015459,0.015506,0.015723,0.015517,None,None


In [39]:
for t in range(0,7):
    print(f"*********************************** TOPIC {t} ***********************************")
    topic = f"topic_{t}"
    for i,articulo in enumerate(data.sort_values('topic_0', ascending=False)['review_body'].values[:5]):
        print(f"review_body #{i}")
        print(articulo[:500])
        print()
    print()

*********************************** TOPIC 0 ***********************************
review_body #0
Estos pendientes son preciosos. Quedan muy elegantes puestos tanto para salir de día como de noche. Tengo más de esta marca, pero al final estos son los que mas me pongo. Son vistosos a la vez que elegantes y pegan con un montón de ropa. También se ponen y quitan facilmente con el gancho que tienen lo que ayuda a que sean uno de mis favoritos. Son de cristal. A mi alguna vez se me han caido al suelo y no les ha pasado nada. No digo que haya que estar tirándolos y tentando la suerte pero los míos

review_body #1
Lo que es la edición coleccionista es muy pobre por lo que costó de salida, mucho dinero para tan poca cosa. Un artbook y un steelbook no justifica el precio de salida. Por no hablar de aquellos que piensan venderlo por barbaridades. Ningún fan debería pagar más del precio original. De verdad que no lo vale y yo me considero muy fan de zelda pero esa la verdad. En cuanto al juego, el r

###  `[3 pts]` Preguntas

- ¿Qué conclusiones puede sacar de los resultados de los últimos tres talleres, usando diferentes modelos pero los mismos datos?

Con el modelo LDA y con 7 número de temas y 25 pases se evidenció con el gráfico que las palabras más comunes en los temas son calidad juego producto precio, sin embargo se evidencia una superposición para los cluters 2 y 4. Al incrementar el número a 10 y 25 pases se incrementan las superposiciones, que puede suponer que no es un buen modelo. 

Con 4 temas y 30 pases, se evidencia que no se superponen los temas, sin embargo no es claro de que habla cada tema. Ya que las palabras más comunes se encuentra en los 4 temas que son calidad, bien, precio. Con 5 temas y 50 pases se ve una gráfica que se presume como un buen modelo. Sin embargo pasandole al modelo el documento "juego", "aretes", "pulsera","calidad" se evidencia que para 5 cluster puede pertencer a 4 de ellos.

Respecto con los anteriores modelo, trabajar con datos etiquetas facilita la clasificación de los nuevos documentos, adicional a que con las medidas de evaluación se puede entender el comportamiento del modelo, no obstante, al no tener etiquetas la tarea es más complicada puesto que con Kmeans, y de acuerdo con la selección de los parametro se pueden generar adecuados o no cluster además que se pueden generar muchos. Respecto a LDA la clasificación de temas facilita en el entendimiento del corpus sin embargo como se notó en el entrenamiento hay palabras que pueden tener la misma posibilidad de estar en diferentes temas, lo que calramento podría afectar el entendiemiento de cada uno de estos.

Julieth Andrea Murte Tavera ⭐